<a href="https://colab.research.google.com/github/Bageshjha/Breast_Cancer_Prediction/blob/main/Breast_Cancer_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Libraries

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'using device : {device}')

using device : cuda


loading the data

In [ ]:
data = load_breast_cancer()
X = data.data
Y = data.target

About the data set-:

---

569 samples with 30 features each.

Two classes: Malignant (0) & Benign (1).



In [ ]:
print(X)


[[1.799e+01 1.038e+01 1.228e+02 ... 2.654e-01 4.601e-01 1.189e-01]
 [2.057e+01 1.777e+01 1.329e+02 ... 1.860e-01 2.750e-01 8.902e-02]
 [1.969e+01 2.125e+01 1.300e+02 ... 2.430e-01 3.613e-01 8.758e-02]
 ...
 [1.660e+01 2.808e+01 1.083e+02 ... 1.418e-01 2.218e-01 7.820e-02]
 [2.060e+01 2.933e+01 1.401e+02 ... 2.650e-01 4.087e-01 1.240e-01]
 [7.760e+00 2.454e+01 4.792e+01 ... 0.000e+00 2.871e-01 7.039e-02]]


In [ ]:
print(Y[:20])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]


Training the data

In [ ]:
X_train , X_test , Y_train , Y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

In [ ]:
print(X.shape,X_train.shape,X_test.shape)

(569, 30) (455, 30) (114, 30)


In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
# converting the data to PyTorch tensors and move it to GPU
X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
Y_train = torch.tensor(Y_train, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
Y_test = torch.tensor(Y_test, dtype=torch.float32).to(device)

Creating the neural network

In [ ]:
class NeuralNet(nn.Module):

  def __init__(self, input_size, hidden_size, output_size):
    super(NeuralNet, self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, output_size)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)
    out = self.sigmoid(out)
    return out

In [ ]:
#defining hyperparameters
input_size = X_train.shape[1]
hidden_size = 64
output_size = 1
learning_rate = 0.001
num_epochs = 100


In [ ]:

# initializing the neural network and moving it to the GPU
model = NeuralNet(input_size, hidden_size, output_size).to(device)

In [ ]:
# defining loss and the optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

Training the model

In [ ]:

for epoch in range(num_epochs):
  model.train()
  optimizer.zero_grad()
  outputs = model(X_train)
  loss = criterion(outputs, Y_train.view(-1,1))
  loss.backward()
  optimizer.step()

  # claculating the accuracy
  with torch.no_grad():
    predicted = outputs.round()
    correct = (predicted == Y_train.view(-1,1)).float().sum()
    accuracy = correct/Y_train.size(0)

  if (epoch+1) % 10 == 0:
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss : {loss.item():.4f}, Accuracy: {accuracy.item() * 100:.2f}%")


Epoch [10/100], Loss : 0.5109, Accuracy: 91.87%
Epoch [20/100], Loss : 0.4041, Accuracy: 92.09%
Epoch [30/100], Loss : 0.3205, Accuracy: 92.97%
Epoch [40/100], Loss : 0.2572, Accuracy: 93.85%
Epoch [50/100], Loss : 0.2109, Accuracy: 94.95%
Epoch [60/100], Loss : 0.1771, Accuracy: 96.04%
Epoch [70/100], Loss : 0.1523, Accuracy: 96.48%
Epoch [80/100], Loss : 0.1338, Accuracy: 96.70%
Epoch [90/100], Loss : 0.1196, Accuracy: 97.36%
Epoch [100/100], Loss : 0.1084, Accuracy: 97.80%


Evaluating the training and test set

In [ ]:
# evaluation on training set
model.eval()
with torch.no_grad():
  outputs = model(X_train)
  predicted = outputs.round()
  correct = (predicted == Y_train.view(-1,1)).float().sum()
  accuracy = correct/Y_train.size(0)
  print(f"Accuracy on training data: {accuracy.item() * 100:.2f}%")

Accuracy on training data: 97.80%


In [ ]:
# evaluation on test set
model.eval()
with torch.no_grad():
  outputs = model(X_test)
  predicted = outputs.round()
  correct = (predicted == Y_test.view(-1,1)).float().sum()
  accuracy = correct/Y_test.size(0)
  print(f"Accuracy on test data: {accuracy.item() * 100:.2f}%")

Accuracy on test data: 98.25%


In [ ]:
# Converting predictions and actual labels to CPU NumPy arrays for printing
X_test_np = X_test.cpu().numpy()
Y_test_np = Y_test.cpu().numpy()
predicted_np = predicted.cpu().numpy().flatten()

In [ ]:
# Print test data, actual labels, and predicted labels
print("\nTest Data (X_test):\n", X_test_np)
print("\nActual Labels (y_test):\n", Y_test_np)
print("\nPredicted Labels:\n", predicted_np)


Test Data (X_test):
 [[-0.46649742 -0.13728933 -0.44421136 ... -0.19435087  0.17275669
   0.20372996]
 [ 1.3653635   0.49866474  1.3055109  ...  0.9917786  -0.561211
  -1.0083895 ]
 [ 0.38006577  0.06921974  0.4041014  ...  0.57035017 -0.10783139
  -0.20629287]
 ...
 [-0.7354724  -0.99852604 -0.7413884  ... -0.2774106  -0.3820785
  -0.32408327]
 [ 0.02898271  2.0334027   0.0274851  ... -0.49027026 -1.6090568
  -0.33137506]
 [ 1.8721689   2.8007715   1.8035499  ...  0.7925579  -0.05868885
  -0.09467243]]

Actual Labels (y_test):
 [1. 0. 0. 1. 1. 0. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 1. 0. 0. 1. 0. 1.
 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1.
 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0.
 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 1. 0. 1. 1. 0.]

Predicted Labels:
 [1. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1.
 1. 1. 1. 1. 1. 0. 1. 1